# Methodologie test utilisée pour résolution operation journaliere



1. Import des données réseaux et noeuds

In [1]:
# import des données globales au problème
from global_parameters import *

from data_import import *
from optimization import *
from utils import *

thermal_grid = import_thermal_data('data/tLoad.csv', 'data/thermalGrid.csv', 'data/thermalNodes.csv')

elec_grid = import_elec_data(
        'data/linePREDIS.csv', 'data/loadprofileP.csv', 'data/loadprofileQ.csv', 'data/ElecprodProfileP.csv')



Set parameter Username
Set parameter LicenseID to value 2756363
Academic license - for non-commercial use only - expires 2026-12-18


2. Création du pb gurobi et résolution

In [2]:
m = create_curt_model(elec_grid,thermal_grid)
m.optimize()

3. Conversion de la solution en dataframe (df)

In [3]:
from gurobipy import GRB
df = df_from_model(m)

## Problème de renforcement du réseau électrique

On crée une fonction _f_ qui sert de fitness function pour le problème de renforcement. Ici la fonciton f renvoie le TOTEX du réseau électrique de distribution a partir d'un choix de renforcement.

X=[0]*9=[0,0,0,0,0,0,0,0,0] correspond au cas de base, aucune des 9 lignes n'est renforcée. Par défaut l'effacement est autorisé et aucun stockage thermique est installé.

f([0]*9) renvoie le TOTEX du réseau non renforcé mais couplé avec une PAC. 

In [4]:
f = make_fitness_TOTEX(elec_grid,thermal_grid,True,False)
print(f([0]*9))

3933603.46881712


La fonction simmulated annealing (recuit simulé) permet d'obtenir le renforcement optimal du réseau en fonction des couts OPEX (pertes Joule & remboursement effacement) et CAPEX (investissement lignes électriques). Avec un couplage de 3MW le renforcement optimal est de ne pas renforcer.

In [5]:
simulated_annealing_memory([0]*9,f)

Initial solution: f=3933603.469, x=[0, 0, 0, 0, 0, 0, 0, 0, 0]
Optimization finished.Total iter = 120
Best solution: f=3933603.469, x=[0, 0, 0, 0, 0, 0, 0, 0, 0]| 00min 26s



([0, 0, 0, 0, 0, 0, 0, 0, 0], 3933603.46881712)

On ré-itere cette étude pour différents couplages de réseau:

In [6]:
f0mw = make_fitness_TOTEX(elec_grid,thermal_grid,HP_MAX_ELEC=0.0000001)
f1mw = make_fitness_TOTEX(elec_grid,thermal_grid,HP_MAX_ELEC=1000000)
f2mw = make_fitness_TOTEX(elec_grid,thermal_grid,HP_MAX_ELEC=2000000)
f3mw = make_fitness_TOTEX(elec_grid,thermal_grid,HP_MAX_ELEC=3000000)

totex0mw=simulated_annealing_memory([0]*9,f0mw)[1]
totex1mw=simulated_annealing_memory([0]*9,f1mw)[1]
totex2mw=simulated_annealing_memory([0]*9,f2mw)[1]
totex3mw=simulated_annealing_memory([0]*9,f3mw)[1]


Initial solution: f=19738245.579, x=[0, 0, 0, 0, 0, 0, 0, 0, 0]
Iter   35 | T=160000.000 | best=11397095.325 | x=[2, 2, 0, 0, 0, 0, 0, 0, 0] | 00min 18s
Iter   71 | T=128000.000 | best=11137630.657 | x=[3, 1, 0, 0, 0, 0, 0, 0, 0] | 00min 34s
Iter   74 | T=128000.000 | best=11137630.657 | x=[1, 1, 0, 0, 0, 0, 0, 0, 0] | 00min 34s
Iter   75 | T=128000.000 | best=11137630.657 | x=[1, 1, 0, 0, 0, 0, 0, 0, 0] | 00min 35s
Iter   76 | T=128000.000 | best=11137630.657 | x=[1, 1, 0, 0, 0, 0, 0, 0, 0] | 00min 36s
Iter   77 | T=128000.000 | best=11137630.657 | x=[1, 1, 0, 0, 0, 0, 0, 0, 0] | 00min 36s
Iter   78 | T=128000.000 | best=11137630.657 | x=[1, 1, 0, 0, 0, 0, 0, 0, 0] | 00min 37s
Iter   79 | T=128000.000 | best=11137630.657 | x=[1, 1, 0, 0, 0, 0, 0, 0, 0] | 00min 37s
Iter   80 | T=128000.000 | best=11137630.657 | x=[1, 1, 0, 0, 0, 0, 0, 0, 0] | 00min 37s
Iter   81 | T=128000.000 | best=11137630.657 | x=[1, 1, 0, 0, 0, 0, 0, 0, 0] | 00min 38s
Iter   82 | T=128000.000 | best=11137630.657 |

In [7]:
print(totex0mw)
print(totex1mw)
print(totex2mw)
print(totex3mw)


11137630.65721432
7400788.584715695
5047078.766153829
3933603.46881712
